In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import timeseries_dataset_from_array
import matplotlib.pyplot as plt
import seaborn as sns
from MultiSeriesWindowsGenerator import MultiSeriesWindowsGenerator
import IPython.display

2023-04-11 13:56:48.146921: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
pd.set_option("display.max_column", 200)

In [3]:
data_list = []
for i in range(0, 27):
    df = pd.read_csv(f"data/aggregated_individual_data_interpolation/interpolation/{i}_interpolated.csv", index_col=0)
    df["subject_id"] = i+1
    data_list.append(df)

# Concatenate the data into a single dataset
data = pd.concat(data_list)
data.drop(["circumplex.arousal_std","circumplex.valence_std","mood_std","activity_std"], inplace=True,axis=1)
# date_time = pd.to_datetime(df.pop('date'))
# df['days'] = date_time
# df = data

In [4]:
use_date = 0

if use_date:
    data.drop(["date"], inplace=True,axis=1)
    date_time = pd.to_datetime(data.pop('date'))
    df = data
    df['days'] = date_time

else: # use days from day 0 of recording
    data.drop(["date"], inplace=True,axis=1)
    df = data

df.reset_index(inplace=True,drop=True)
df = df.astype({'subject_id': 'float64','days': 'float64','weekday': 'float64'})

In [5]:
df = data
df.reset_index(inplace=True,drop=True)
df = df.astype({'subject_id': 'float64','days': 'float64','weekday': 'float64'})

In [6]:
LABELS = ['mood']
REGRESSORS = ['weekday', 'circumplex.arousal', 'circumplex.valence',
       'activity', 'screen', 'call', 'sms', 'appCat.builtin',
       'appCat.communication', 'appCat.entertainment', 'appCat.finance',
       'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social',
       'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']

DATE = 'days' # always correct
IN_STEPS = 7 # use 7 days
OUT_STEPS = 7 # to predict 1 day in the future
GROUPBY = ['subject_id']
BATCH_SIZE = 8

In [7]:
n = len(df)
train_series = df.groupby(GROUPBY, as_index=False, group_keys=False).apply(lambda x: x.iloc[:int(len(x) * 0.7)]).reset_index(drop=True)
val_series = df.groupby(GROUPBY, as_index=False, group_keys=False).apply(lambda x: x.iloc[int(len(x) * 0.7):int(len(x) * 0.9)]).reset_index(drop=True)
test_series = df.groupby(GROUPBY, as_index=False, group_keys=False).apply(lambda x: x.iloc[int(len(x) * 0.9):]).reset_index(drop=True)
train_series.shape, val_series.shape, test_series.shape

((804, 22), (234, 22), (130, 22))

In [8]:
test_window = MultiSeriesWindowsGenerator(
    input_width=IN_STEPS, label_width=OUT_STEPS, shift=1, batch_size=BATCH_SIZE, GROUPBY=GROUPBY, label_columns=LABELS, regressor_columns=REGRESSORS, DATE=DATE, LABELS=LABELS)

In [16]:
test_window.train_df

AttributeError: 'MultiSeriesWindowsGenerator' object has no attribute 'train_df'

In [14]:
print(test_series[test_series["subject_id"]==1][["weekday","days"]])

   weekday  days
0      2.0  40.0
1      3.0  41.0
2      4.0  42.0
3      5.0  43.0
4      6.0  44.0


In [15]:
test = preprocess_dataset_test(test_window, data=test_series)
test

weekday                                                         \
days          27.0 29.0 30.0 31.0 32.0 33.0 34.0 35.0 36.0 37.0 38.0 39.0   
subject_id                                                                  
1.0            NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2.0            NaN  NaN  NaN  2.0  3.0  NaN  5.0  NaN  NaN  1.0  NaN  NaN   
3.0            NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4.0            NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
5.0            NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  4.0   
6.0            NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
7.0            NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
8.0            NaN  NaN  NaN  NaN  NaN  2.0  3.0  4.0  5.0  NaN  NaN  NaN   
9.0            NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  4.0  5.0  6.0  0.0   
10.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
11.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
12.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
13.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
14.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
15.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
16.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  5.0  6.0  0.0   
17.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  6.0  0.0  1.0  2.0   
18.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
19.0           0.0  2.0  3.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
20.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
21.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  5.0  6.0  0.0   
22.0           NaN  NaN  NaN  NaN  NaN  NaN  6.0  0.0  NaN  2.0  3.0  NaN   
23.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  5.0  6.0   
24.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
25.0           NaN  NaN  NaN  5.0  6.0  0.0  NaN  2.0  NaN  NaN  NaN  NaN   
26.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  4.0  5.0   
27.0           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

                                                                             \
days       40.0 41.0 42.0 43.0 44.0 45.0 46.0 47.0 48.0 49.0 50.0 51.0 52.0   
subject_id                                                                    
1.0         2.0  3.0  4.0  5.0  6.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2.0         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3.0         NaN  NaN  4.0  5.0  6.0  0.0  NaN  2.0  NaN  NaN  NaN  NaN  NaN   
4.0         NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0  3.0  4.0  5.0  6.0  0.0   
5.0         5.0  6.0  0.0  NaN  2.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6.0         NaN  NaN  2.0  3.0  4.0  5.0  6.0  NaN  NaN  NaN  NaN  NaN  NaN   
7.0         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0  3.0  4.0  5.0  6.0   
8.0         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
9.0         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
10.0        NaN  NaN  NaN  NaN  NaN  NaN  0.0  1.0  2.0  3.0  4.0  5.0  NaN   
11.0        NaN  3.0  4.0  5.0  6.0  0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
12.0        NaN  NaN  NaN  NaN  NaN  NaN  1.0  2.0  3.0  NaN  5.0  NaN  0.0   
13.0        NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  2.0  3.0  4.0  NaN  6.0   
14.0        2.0  3.0  4.0  5.0  NaN  0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
15.0        NaN  3.0  4.0  5.0  6.0  0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
16.0        1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
17.0        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
18.0        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  2.0  3.0   
19.0        NaN  NaN  NaN  NaN  NaN  NaN 

# Baseline Model

In [ ]:
cur_window = MultiSeriesWindowsGenerator(
    input_width=IN_STEPS, label_width=OUT_STEPS, shift=1, batch_size=BATCH_SIZE, GROUPBY=GROUPBY, label_columns=LABELS, regressor_columns=REGRESSORS, DATE=DATE, LABELS=LABELS)
cur_window.update_datasets(train_series, val_series, test_series, norm=True)

In [ ]:
cur_window.train_df

In [ ]:
class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

In [ ]:
baseline = Baseline(label_index=cur_window.column_indices[LABELS[0]])

baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                 metrics=[tf.keras.metrics.MeanAbsoluteError()])

val_performance = {}
performance = {}
val_performance['Baseline'] = baseline.evaluate(cur_window.val)
performance['Baseline'] = baseline.evaluate(cur_window.test, verbose=0)

In [ ]:
def plot_2(self, model=None, plot_col=None, max_subplots=3):
    inputs, labels = self.example
    if not plot_col:
        plot_col = self.LABELS[0]
    plt.figure(figsize=(12, 8))
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))

    for n in range(max_n):
        plt.subplot(max_n, 1, n + 1)
        plt.ylabel(f'{plot_col} {"[normed]" if self.norm else ""}')
        plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                 label='Inputs', marker='.', zorder=-10)

        if self.label_columns:
            label_col_index = self.label_columns_indices.get(plot_col, None)
        else:
            label_col_index = plot_col_index

        if label_col_index is None:
            continue

        plt.scatter(self.label_indices, labels[n, :, label_col_index],
                    edgecolors='k', label='Labels', c='#2ca02c', s=64)
        if model is not None:
            predictions = model(inputs)
            print(self.label_indices, predictions[n, :, label_col_index])
            plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                        marker='X', edgecolors='k', label='Predictions',
                        c='#ff7f0e', s=64)

        if n == 0:
            plt.legend()

    plt.xlabel('Time [Days]')
    plt.show()

In [ ]:
cur_window.plot(baseline)

In [ ]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [ ]:
print('Input shape:', cur_window.example[0].shape)
print('Output shape:', lstm_model(cur_window.example[0]).shape)

In [ ]:
history = compile_and_fit(lstm_model, cur_window)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate(cur_window.val)
performance['LSTM'] = lstm_model.evaluate(cur_window.test, verbose=0)

In [ ]:
wide_window.plot(lstm_model)

# RNN

#### define window


In [ ]:
LABELS = ['mood']
REGRESSORS = ['weekday', 'circumplex.arousal', 'circumplex.valence',
       'activity', 'screen', 'call', 'sms', 'appCat.builtin',
       'appCat.communication', 'appCat.entertainment', 'appCat.finance',
       'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social',
       'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']

DATE = 'days' # always correct
IN_STEPS = 7 # use 7 days
OUT_STEPS = 1 # to predict 1 day in the future
GROUPBY = ['subject_id']
BATCH_SIZE = 7

In [ ]:
wide_window = MultiSeriesWindowsGenerator(
    input_width=IN_STEPS, label_width=OUT_STEPS, shift=OUT_STEPS, batch_size=BATCH_SIZE, GROUPBY = GROUPBY,
    label_columns=LABELS, regressor_columns=REGRESSORS, DATE=DATE, LABELS=LABELS)
wide_window.update_datasets(train_series, val_series, test_series, norm=True)
wide_window

#### training procedure

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [ ]:
print('Input shape:', cur_window.example[0].shape)
print('Output shape:', lstm_model(cur_window.example[0]).shape)

In [ ]:
import IPython.display

In [ ]:
cur_window.test_df

In [ ]:
history = compile_and_fit(lstm_model, cur_window)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate(cur_window.val)
performance['LSTM'] = lstm_model.evaluate(cur_window.test, verbose=0)

In [ ]:
wide_window.plot(lstm_model)

# Autoregressive RNN